In [55]:
import pandas as pd
import numpy as np

In [ ]:
# train = pd.read_csv("output/sorted_train0310.csv")
# test = pd.read_csv("output/sorted_test0310.csv")
# val = pd.read_csv("output/sorted_val0310.csv")

In [56]:
train = pd.read_csv("output/random_train0310.csv")
test = pd.read_csv("output/random_test0310.csv")
val = pd.read_csv("output/random_val0310.csv")

In [57]:
# Compute some statistics
chargeoff_count = (train['LoanStatus'] == "CHGOFF").sum()
print 'Chargeoffs:', chargeoff_count
print 'Percentage of loans that defaulted:', float(chargeoff_count)/len(train)

chargeoff_count = (test['LoanStatus'] == "CHGOFF").sum()
print 'Chargeoffs:', chargeoff_count
print 'Percentage of loans that defaulted:', float(chargeoff_count)/len(test)

Chargeoffs: 7124
Percentage of loans that defaulted: 0.162481468811
Chargeoffs: 891
Percentage of loans that defaulted: 0.162561576355


In [40]:
####### optional ##########
# if import from sorted data - need to downsample train dataset to be more balanced
# can call the following function multiple times for decreasing in log term
# dropping PIF samples with 50% probability

def drop_half(train):
    print 'before drop: ', train.shape[0]
    dropidx = []
    for idx, val in train['LoanStatus'].iteritems():
        if val != 'CHGOFF':
            rand = np.random.rand()
            if rand < 0.5:
                dropidx.append(idx)
    train = train.drop(dropidx)
    print 'after drop: ', train.shape[0]
    return train
    
train = drop_half(train)
train = drop_half(train)
# test = drop_half(test)

chargeoff_count = (train['LoanStatus'] == "CHGOFF").sum()
print 'Chargeoffs:', chargeoff_count
print 'Percentage of loans that defaulted:', float(chargeoff_count)/len(train)

chargeoff_count = (test['LoanStatus'] == "CHGOFF").sum()
print 'Chargeoffs:', chargeoff_count
print 'Percentage of loans that defaulted:', float(chargeoff_count)/len(test)

before drop:  43845
after drop:  25440
before drop:  25440
after drop:  16330
Chargeoffs: 7124
Percentage of loans that defaulted: 0.436252296387
Chargeoffs: 891
Percentage of loans that defaulted: 0.162561576355


In [58]:
# filtering for the columns you don't want to include for your model
c = train.columns.tolist()
c.remove('LoanStatus')
c.remove('ChargeOffDate')
c.remove('GrossChargeOffAmount')
c.remove('BorrZip')
c.remove('CDC_Zip')
c.remove('BorrCity')
c.remove('CDC_City')
c.remove('ThirdPartyLender_City')
# c.remove('BorrState')
# c.remove('CDC_State')
# c.remove('ThirdPartyLender_State')
c.remove('ProjectCounty')
c.remove('ProjectState')
c.remove('ApprovalDate')
print c

['BorrState', 'CDC_State', 'ThirdPartyLender_State', 'ApprovalFiscalYear', 'DeliveryMethod', 'subpgmdesc', 'TermInMonths', 'BusinessType', 'SP500_Yearly_Return', 'CPI', 'Log_GrossApproval_Norm', 'Log_HPI_Norm', 'ThirdPartyDollars_Norm', 'TermMultipleYear', 'RepeatBorrower', 'BankStateneqBorrowerState', 'ProjectStateneqBorrowerState', '2DigitNaics']


In [59]:
x_train = train[c]
x_test = test[c]

y_train_logit = train['LoanStatus']
y_test_logit = test['LoanStatus']

# hazard_target = train['ChargeOffDate']
# amount_target = train['GrossChargeOffAmount']

# print train.dtypes
# convert categorical variables to dummy variables
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)
# val = pd.get_dummies(val)

In [60]:
# to get consistent feature dimensions for both train and test dataset
def add_missing_dummy_columns(d1, d2):
    missing_cols = set(d1.columns) - set(d2.columns)
    for c in missing_cols:
        d2[c] = 0
    return d2

print 'before fix columns: '
print x_train.shape
print x_test.shape

def fix_columns(x_train, x_test):  

    x_test = add_missing_dummy_columns(x_train, x_test)

    extra_cols = set(x_test.columns) - set(x_train.columns)
    x_test = x_test[x_train.columns]
    return x_test

x_test = fix_columns(x_train, x_test)
print 'after fix columns: '
print x_train.shape
print x_test.shape

before fix columns: 
(43845, 216)
(5481, 208)
after fix columns: 
(43845, 216)
(5481, 216)


In [61]:
# fit to models 
# below is to fit to logit model

# y_train_logit = np.reshape(l, (len(l),))
from sklearn.linear_model import LogisticRegression
solver = 'saga'

lr = LogisticRegression(solver=solver, 
                        multi_class= 'multinomial', 
                        C=1,
                        penalty = 'l1',
                        fit_intercept=True,
                        max_iter=10,
                        random_state=42, 
                       )
lr.fit(x_train, y_train_logit)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10, multi_class='multinomial',
          n_jobs=1, penalty='l1', random_state=42, solver='saga',
          tol=0.0001, verbose=0, warm_start=False)

In [62]:
lr.decision_function(x_train)
print lr.coef_

[[  1.14012683e-03   3.91185989e-04  -5.70427979e-03   9.02913310e-03
    1.86245702e-02   3.43711503e-02   3.41452563e-02  -6.21555152e-05
    1.00078378e-03   2.37574882e-03  -1.19283228e-04  -8.46344244e-06
    4.25962877e-04   1.88892192e-04   5.69593025e-04  -3.82150009e-03
    1.44969906e-04   1.16820604e-04  -5.93008856e-05  -2.29988419e-05
    2.65389596e-03   1.23065381e-03  -5.78667480e-06  -1.86651582e-04
   -7.79204810e-05  -5.07594664e-05   8.79141850e-04   2.49103004e-05
   -2.44341223e-07   1.21987279e-04  -1.96834895e-04  -3.63328630e-04
   -2.24215304e-04   1.53777112e-04   9.96089864e-04   1.16009278e-04
   -4.60057493e-05   1.69528458e-04  -1.72244998e-04  -2.53156096e-04
   -3.02486773e-04   2.38987466e-05  -5.51799175e-04   8.55789286e-05
   -1.37790457e-04   2.51129865e-04  -7.87796069e-04  -4.65526903e-04
    2.73986620e-04  -2.92577790e-04   3.96661117e-06   2.40316281e-04
   -1.05563052e-04   1.64506240e-04  -6.57713059e-07   2.47694358e-04
    7.86040888e-04  

In [63]:
# fit on training data to see if there is underfitting
y_pred_train = lr.predict(x_train)
print 'train: ', np.sum(y_pred_train == y_train_logit) *1./ y_train_logit.shape[0]

# fit on test set to see if there is overfitting
y_pred_test = lr.predict(x_test)
print 'test: ', np.sum(y_pred_test == y_test_logit) *1./ y_test_logit.shape[0]

train:  0.837450108336
test:  0.837620872104


In [64]:
from sklearn import metrics

print metrics.classification_report(y_test_logit, y_pred_test)

             precision    recall  f1-score   support

     CHGOFF       1.00      0.00      0.00       891
        PIF       0.84      1.00      0.91      4590

avg / total       0.86      0.84      0.76      5481

